In [1]:
!pip install tensorflow==2.19.0

In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
import os
import random
import pathlib

In [3]:
image_dir = "C:/Users/HP/Desktop/ML_DS/CNN/data/oxford-iiit-pet/images/images"
label_file = "C:/Users/HP/Desktop/ML_DS/CNN/data/oxford-iiit-pet/annotations/annotations/list.txt"

with open(label_file, 'r') as f:
    lines = f.readlines()[6:] 

data = []
for line in lines:
    parts = line.strip().split()
    if len(parts) >= 2:
        filename = parts[0] + ".jpg"
        label = "_".join(parts[0].split("_")[:-1])
        full_path = os.path.join(image_dir, filename)
        if os.path.exists(full_path):
            data.append((full_path, label))


In [4]:
label_names = sorted(set(label for _, label in data))
label2idx = {label: idx for idx, label in enumerate(label_names)}
data = [(img, label2idx[label]) for img, label in data]

train_val_data, test_data = train_test_split(data, test_size=0.15, stratify=[label for _, label in data], random_state=42)
train_data, val_data = train_test_split(train_val_data, test_size=0.15, stratify=[label for _, label in train_val_data], random_state=42)

def build_dataset(data_list):
    paths, labels = zip(*data_list)
    ds = tf.data.Dataset.from_tensor_slices((list(paths), list(labels)))

    def load_and_preprocess(path, label):
        image = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (224, 224))
        image = preprocess_input(image)
        return image, label

    return ds.map(load_and_preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

train_ds = build_dataset(train_data)
val_ds = build_dataset(val_data)
test_ds = build_dataset(test_data)

In [5]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.05),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomTranslation(0.05, 0.05),
])

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)

train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)


In [6]:
num_classes = len(label2idx)
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,096,968 (15.63 MB)

 Trainable params: 47,397 (185.14 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [8]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
os.makedirs("augmented_model", exist_ok=True)
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True, monitor='val_loss'),
    ModelCheckpoint("augmented_model/best_model.h5", save_best_only=True, monitor='val_loss')
]

In [11]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=30,
          callbacks=callbacks)

Epoch 1/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 654ms/step - accuracy: 0.5118 - loss: 2.1248

166/166 ━━━━━━━━━━━━━━━━━━━━ 137s 777ms/step - accuracy: 0.5130 - loss: 2.1200 - val_accuracy: 0.8922 - val_loss: 0.4841
Epoch 2/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 658ms/step - accuracy: 0.8792 - loss: 0.4928

166/166 ━━━━━━━━━━━━━━━━━━━━ 130s 778ms/step - accuracy: 0.8792 - loss: 0.4926 - val_accuracy: 0.9082 - val_loss: 0.3433
Epoch 3/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 652ms/step - accuracy: 0.9109 - loss: 0.3492

166/166 ━━━━━━━━━━━━━━━━━━━━ 127s 763ms/step - accuracy: 0.9109 - loss: 0.3491 - val_accuracy: 0.9136 - val_loss: 0.3017
Epoch 4/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 647ms/step - accuracy: 0.9262 - loss: 0.2815

166/166 ━━━━━━━━━━━━━━━━━━━━ 127s 758ms/step - accuracy: 0.9262 - loss: 0.2815 - val_accuracy: 0.9157 - val_loss: 0.2789
Epoch 5/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 646ms/step - accuracy: 0.9339 - loss: 0.2475

...

166/166 ━━━━━━━━━━━━━━━━━━━━ 124s 745ms/step - accuracy: 0.9437 - loss: 0.1934 - val_accuracy: 0.9210 - val_loss: 0.2583
Epoch 8/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 627ms/step - accuracy: 0.9496 - loss: 0.1754

166/166 ━━━━━━━━━━━━━━━━━━━━ 123s 735ms/step - accuracy: 0.9496 - loss: 0.1754 - val_accuracy: 0.9189 - val_loss: 0.2567
Epoch 9/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 633ms/step - accuracy: 0.9546 - loss: 0.1644

166/166 ━━━━━━━━━━━━━━━━━━━━ 124s 741ms/step - accuracy: 0.9546 - loss: 0.1644 - val_accuracy: 0.9178 - val_loss: 0.2561
Epoch 10/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 633ms/step - accuracy: 0.9538 - loss: 0.1530

166/166 ━━━━━━━━━━━━━━━━━━━━ 124s 743ms/step - accuracy: 0.9538 - loss: 0.1531 - val_accuracy: 0.9168 - val_loss: 0.2557
Epoch 11/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 633ms/step - accuracy: 0.9585 - loss: 0.1489

166/166 ━━━━━━━━━━━━━━━━━━━━ 124s 743ms/step - accuracy: 0.9585 - loss: 0.1489 - val_accuracy: 0.9157 - val_loss: 0.2477
Epoch 12/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 124s 742ms/step - accuracy: 0.9634 - loss: 0.1358 - val_accuracy: 0.9178 - val_loss: 0.2500
Epoch 13/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 123s 737ms/step - accuracy: 0.9600 - loss: 0.1352 - val_accuracy: 0.9210 - val_loss: 0.2495
Epoch 14/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 123s 735ms/step - accuracy: 0.9683 - loss: 0.1180 - val_accuracy: 0.9232 - val_loss: 0.2490
